# Actor-Critic methods

The REINFORCE algorithm is generally implemented as an episodic algorithm,
meaning that we only apply it to update our model parameters after the agent has
completed an entire episode (and collected rewards along the way).
By sampling a full episode, we get a pretty good idea of the true value of an action
because we can see its downstream effects rather than just its immediate effect (which
may be misleading due to randomness in the environment); this full episode sampling
is under the umbrella of Monte Carlo approaches.  The advantage of this method is that
it’s unbiased. Since we’re not estimating the return, we use only the true return we
obtain. However, given the stochasticity of the environment (random events during an
episode) and stochasticity of the policy, trajectories can lead to different returns,
which can lead to high variance. Consequently, the same starting state can lead to very
different returns. Because of this, the return starting at the same state can vary
significantly across episodes.

Remember that the policy gradient estimation is the direction of the steepest increase in return. In other words, how to update our policy weights so that actions that lead to good returns have a higher probability of being taken. The Monte Carlo variance, which we will further study in this unit, leads to slower training since we need a lot of samples to mitigate it.

There are two challenges we want to overcome to
increase the robustness of the policy learner:
- We want to improve the sample efficiency by updating more frequently.
- We want to decrease the variance of the reward we used to update our model.

These problems are related, since the reward variance depends on how many samples
we collect (more samples yields less variance). The idea behind a combined value-policy
algorithm is to use the value learner to reduce the variance in the rewards that are
used to train the policy.

To address these challenges, we introduce **Actor-Critic** methods, a hybrid architecture combining value-based and Policy-Based methods that helps to stabilize the training by reducing the variance using:

- An _Actor_, represented by a function $\pi_\theta(s)$ that controls how our agent behaves (Policy-Based method)
- A _Critic_, represented by the function $q_w(s,a)$ that measures how good the taken action is (Value-Based method)

The Actor-Critic process works like this:
1. The Actor (policy) takes the current state $s_t$ and outputs an action $a_t$.
2. The Critic takes that $a_t$ and $s_t$ as inputs to compute the Q-value.
3. The action $A_t$ performed in the environment outputs a new state $s_{t+1}$ and a
   reward $r_{t+1}$.
4. The Actor updates its policy parameters using the Q value.

<figure>
    <img src="step4_ac.png" alt="Caption" width="800" />
    <figcaption>Update of the policy paramters (taken from huggingface.co).</figcaption>
</figure>

5. Thanks to its updated parameters, the Actor produces the next action $A_{t+1}$ to
   take given the new state $S_{t+1}$.
6. The Critic then updates its value parameters.

<!-- <figure>
    <img src="step5_ac.png" alt="Caption" width="800" />
    <figcaption>Policy gradient objective function (taken from huggingface.co).</figcaption>
</figure> -->

We can stabilize learning further by using the **Advantage Function** $A(s,a)$ instead
of the action-value function in Actor-Critic. We call this method **Advantage Actor
Critic** (A2C). The Advantage Function calculates the relative advantage of an action compared to
the others
possible at a state:

$$ A(s,a) = Q(s,a) - V(s) $$

with $V(s)$ the state-value function.  In other words, this function calculates the extra reward we get if we take this action at that state compared to the mean reward we get at that state.

- If $A(s,a) > 0$: our gradient is pushed in that direction.
- If $A(s,a) < 0$ (our action does worse than the average value of that state), our gradient is pushed in the opposite direction.

To avoid using both $Q(s,a)$ and $V(s)$, we can estimate $Q(s,a)$ as $r_{t+1} + \gamma
V(s_{t+1})$, with $\gamma$ the discount factor.

We will train a neural network to predict both the policy and the action-value or the state-value.

<figure>
    <img src="ac_scheme.png" alt="Caption" width="800" />
    <figcaption>The general overview of actor-critic methods (taken from "Deep Reinforcement Learning in Action")<figcaption>
</figure>

### Proximal Policy Optimization (PPO)

The idea with Proximal Policy Optimization (PPO) is that we want to improve the training stability of the policy by limiting the change you make to the policy at each training epoch: we want to avoid having too large of a policy update.

To do so, we need to measure how much the current policy changed compared to the former one using a ratio calculation between the current and former policy. And we clip this ratio in a range
$[1−\epsilon,1+\epsilon]$, meaning that we remove the incentive for the current policy
to go too far from the old one (hence the proximal policy term).

You can read more about this [here](https://huggingface.co/learn/deep-rl-course/unit8/clipped-surrogate-objective).

## Using Stable Baselines JAX (SBX)

In [ ]:
import gymnasium as gym

from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from stable_baselines3.common.logger import configure

In [ ]:
# CartPole-v1, LunarLander-v2, Pendulum-v1, Acrobot-v1, Reacher-v2, Ant
environment = "Ant"
params_cartpole = {}
params_pendulum = {"learning_rate": 1e-3}
params_lunarlander = {"learning_rate": 1e-3} # total_timesteps=5e5
params_acrobot = {"learning_rate": 1e-3, "gamma": 0.99, "gae_lambda": 0.94} # total_timesteps = 5e5
    # "n_steps": 512,
    # "batch_size": 64,
    # "n_epochs":5,
    # "gamma":0.999,
    # "gae_lambda":0.98,
    # "ent_coef":0.01} # total_timesteps = 5e5

In [ ]:
def train(env_name, algo_params, total_timesteps=10000, progress_bar=True):
    env = gym.make(env_name)
    tmp_path = "tmp/sb3_log/"
    # set up logger
    new_logger = configure(tmp_path, ["csv"])

    model = PPO("MlpPolicy", env, **algo_params)
    # Set new logger
    model.set_logger(new_logger)
    model.learn(total_timesteps=total_timesteps, progress_bar=progress_bar)

    # vec_env = model.get_env()
    # obs = vec_env.reset()
    # total_reward = 0.
    # for _ in range(1000):
    #     # vec_env.render()
    #     action, _states = model.predict(obs, deterministic=True)
    #     obs, reward, _, _ = vec_env.step(action)
    #     total_reward += reward

    # vec_env.close()
    return model

In [ ]:
model = train(environment, params_cartpole, total_timesteps=5e5)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("tmp/sb3_log/progress.csv")


In [ ]:
ep_rew_mean = df["rollout/ep_rew_mean"].to_numpy()
plt.plot(ep_rew_mean)
plt.show()

In [ ]:
import numpy as np

def play(env_name, model, num_games=1, max_actions=1000, render=False):
    if render:
        env = gym.make(env_name, render_mode="human")
    else:
        env = gym.make(env_name)

    total_rewards = np.zeros(num_games)
    for i in range (num_games):
        total_reward = 0
        obs = np.array(env.reset()[0])
        for _ in range(max_actions):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            obs = np.array(obs)
            if render:
                env.render()
            if terminated or truncated:
                break
        total_rewards[i] = total_reward

    print("Average total reward: ", np.mean(total_rewards))

In [ ]:
play(environment, model, num_games=1000)

In [ ]:
play(environment, model, num_games=1, render=True)